In [210]:
import tensorstore as ts
import numpy as np
import jax
import jax.numpy as jnp
from chex import Array

In [94]:
dataset = ts.open(
    {
        'driver': 'zarr',
        'kvstore': {
            'driver': 'ocdbt',
            'base': 'file:///Users/callum/Mava/tmp',
            'path': 'obs'
        },
        # 'metadata': {
        #     'dtype': '<u2',
        #     'shape': [1, 10000],
        # },
    },
    dtype=ts.uint32,
    #shape=[100, 100, 100],
    create=True,
).result()

ValueError: Error opening "zarr" driver: Error writing ".zarray" in OCDBT database at local file "/Users/callum/Mava/tmp/": Cannot create using specified "metadata" and schema: domain must be specified [source locations='tensorstore/driver/zarr/driver.cc:449\ntensorstore/driver/zarr/driver.cc:449\ntensorstore/kvstore/kvstore.cc:377\ntensorstore/driver/driver.cc:109'] [tensorstore_spec='{\"context\":{\"cache_pool\":{},\"data_copy_concurrency\":{},\"file_io_concurrency\":{},\"file_io_sync\":true,\"ocdbt_coordinator\":{}},\"create\":true,\"driver\":\"zarr\",\"dtype\":\"uint32\",\"kvstore\":{\"base\":{\"driver\":\"file\",\"path\":\"/Users/callum/Mava/tmp/\"},\"driver\":\"ocdbt\"}}']

In [76]:
await dataset[10:100,10,10].read()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=uint32)

In [139]:
from typing import NamedTuple
import flashbax as fbx

class Timestep(NamedTuple):
    obs: jnp.ndarray
    acts: jnp.ndarray
    rwds: jnp.ndarray

In [140]:
buffer = fbx.make_flat_buffer(
    max_length=1_000_000,
    min_length=1,
    sample_batch_size=1,
)

/Users/callum/miniconda3/envs/mava/lib/python3.9/site-packages/flashbax/buffers/trajectory_buffer.py:473: UserWarning: Setting max_size dynamically sets the `max_length_time_axis` to be `max_size`//`add_batch_size = 1000000`.This allows one to control exactly how many timesteps are stored in the buffer.Note that this overrides the `max_length_time_axis` argument.
  warnings.warn(


In [142]:
state = buffer.init(
   Timestep(
       obs=jnp.zeros((10, 16), dtype=jnp.float32),
       acts=jnp.zeros((10, 32), dtype=jnp.float32),
       rwds=jnp.zeros((10, 64), dtype=jnp.float32),
   )
)

In [211]:
jax.tree_util.tree_map_with_path(lambda path, x: print(path[0].name, (x.shape, x.dtype)), state.experience)

obs ((1, 1000000, 10, 16), dtype('float32'))
acts ((1, 1000000, 10, 32), dtype('float32'))
rwds ((1, 1000000, 10, 64), dtype('float32'))


Timestep(obs=None, acts=None, rwds=None)

In [255]:
import os
DRIVER = "file://"

class Vault:
    def __init__(self, base_path: str) -> None:
        self.base_path = base_path
        if not os.path.isabs(base_path):
            self.base_path = os.path.abspath(base_path)

    def _get_base_spec(self):
        return {
            'driver': 'zarr',
            'kvstore': {
                'driver': 'ocdbt',
                'base': f'{DRIVER}{self.base_path}',
            },
        }
    
    def _write_leaf(self, name: str, leaf: Array, interval: slice = slice(None)):
        spec = self._get_base_spec()
        spec['kvstore']['path'] = name
        leaf_ds = ts.open(
            spec,
            dtype=leaf.dtype,
            shape=leaf[:, interval, ...].shape,
            #create=True,
        ).result()
        leaf_ds[:, interval, ...].write(leaf).result()

    def _read_leaf(self, name: str, interval: slice = slice(None)):
        spec = self._get_base_spec()
        spec['kvstore']['path'] = name
        leaf_ds = ts.open(
            spec,
        ).result()
        return leaf_ds[:, interval, ...].read().result()

    def write(self, state, interval: slice = slice(None)):
        jax.tree_util.tree_map_with_path(
            lambda path, x: self._write_leaf(path[0].name, x, interval),
            state.experience,
        )

    def read(self, interval: slice = slice(None)):
        return jax.tree_util.tree_map_with_path(
            lambda path, _: self._read_leaf(path[0].name, interval),
            state.experience,
        )


In [256]:
v = Vault('vault_test')

In [257]:
v.write(state, slice(0, 10))

ValueError: Error aligning dimensions: source dimension 1 [0, 1000000) mismatch with target dimension 1 [0, 10) [source locations='tensorstore/index_space/alignment.cc:134\ntensorstore/index_space/alignment.cc:169\ntensorstore/driver/write.cc:244']

In [218]:
res = v.read()

In [267]:
interval = slice(0, 10)

SyntaxError: invalid syntax (281763484.py, line 2)

In [241]:
jax.tree_map(lambda x: x[:, 0:1, ...], state.experience)

Timestep(obs=Array([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]]]], dtype=float32), acts=Array([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0

In [252]:
def f(sl):
    print(jnp.arange(10)[sl])

In [254]:
f(slice(None))

[0 1 2 3 4 5 6 7 8 9]
